# STAGE IV - Upload Trained Model to Model Registry

The model has been trained on the new dataset. It is now time to upload the resulting checkpoint to the Model Registry for deployment

In [ ]:
%pip -q install -U pip
%pip -q install model-registry boto3

In [ ]:
from model_registry import ModelRegistry
from model_registry.utils import S3Params
from model_registry.exceptions import StoreError
import os

In [ ]:
# declare global setting variables
PERSISTENCE_DIR: str = os.environ.get('PERSISTENCE_DIR')
CHECKPOINT_NAME: str = os.environ.get("YOLO_CHECKPOINT")
USER: str = os.environ.get("USERNAME")
CLUSTER_DOMAIN: str = os.environ.get("CLUSTER_DOMAIN")
VERSION: str = os.environ.get("VERSION")
BUCKET: str = os.environ.get("BUCKET")
MODEL_NAME: str = os.environ.get("MODEL_NAME")
JOB = os.environ.get("JOB_TYPE", "detect")
RUN_NAME = os.environ.get("JOB_NAME", "train")

In [ ]:
# load results
from pathlib import PosixPath
results_path: str = max(PosixPath(f"{PERSISTENCE_DIR}/runs/{JOB}/").glob("train*"), key=os.path.getmtime)

print(f"Latest training output at: {results_path}")

# make sure checkpoint exists
latest_checkpoint: str = f"{results_path}/weights/best.onnx"

# validate
if not os.path.exists(latest_checkpoint):
    raise Exception(f"Checkpoint {latest_checkpoint} not found in filesystem")

In [ ]:
# Set up the model registry connection
model_registry_url = f"https://registry-rest.{CLUSTER_DOMAIN}"
author_name = USER

print(model_registry_url)

registry = ModelRegistry(server_address=model_registry_url, port=443, author=author_name, is_secure=False)

In [ ]:
# Model details we want to register
registered_model_name = MODEL_NAME
s3_model_bucket = BUCKET
s3_model_prefix = f"{s3_model_bucket}/{registered_model_name}"
version = "1"
onnx_model = f"{latest_checkpoint}"

AWS_S3_ENDPOINT = os.environ.get('AWS_S3_ENDPOINT')
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
AWS_S3_BUCKET = os.environ.get('AWS_S3_BUCKET')
AWS_DEFAULT_REGION = os.environ.get('AWS_DEFAULT_REGION')

# remote S3 paths
s3_region = AWS_DEFAULT_REGION
s3_onnx = f"{s3_model_prefix}/onnx/{registered_model_name}"
minio_endpoint = f"https://minio-s3-s3-minio-dev.{CLUSTER_DOMAIN}"

In [ ]:
# upload parameters for s3 connections
s3_upload_params_onnx = S3Params(
    bucket_name=os.environ.get('AWS_S3_BUCKET'),
    s3_prefix=f"{s3_onnx}/{version}",
)

In [ ]:
# artifact update function
def update_artifact(model_name, model_version, new_uri, storage_path):
    artifact = registry.get_model_artifact(model_name, model_version)
    print(f"Got Artifact {artifact.name} with ID: {artifact.id}\n Current URI: {artifact.uri}\n Updating with URI: {new_uri}\n Current StoragePath: {artifact.storage_path}")
    artifact.uri = new_uri
    artifact.storage_path = storage_path
    registry.update(artifact)

In [ ]:
# upload function
def register(model_name, data_path,
             model_format_name, author, model_format_version,
             model_version, storage_path, description,
             metadata, upload_parms):
    try:
        # register onnx model
        registered_model = registry.upload_artifact_and_register_model(
            name=model_name,
            model_files_path=data_path,
            model_format_name=model_format_name,
            author=author,
            model_format_version=model_format_version,
            version=model_version,
            storage_path=storage_path,
            description=description,
            metadata=metadata,
            upload_params=upload_parms
        )
        print(f"'{model_name}' version '{model_version}'\n URL: https://rhods-dashboard-redhat-ods-applications.{CLUSTER_DOMAIN}/modelRegistry/registry/registeredModels/1/versions/{registry.get_model_version(model_name, model_version).id}/details")
    except StoreError:
        stored_version = registry.get_model_version(registered_model_name, f"{version}-onnx")
        print(f"Model version {stored_version.name}-{stored_version.id} already exists: Updating URI...")
        new_uri = f"s3://{storage_path}?endpoint={minio_endpoint}&defaultRegion={s3_region}"
        update_artifact(model_name, model_version, new_uri, storage_path)

In [ ]:
# data to register in the model registry
models = [
    {
        "model_name": registered_model_name,
        "data_path": onnx_model,
        "author": USER,
        "model_format_name": "onnx",
        "model_format_version": VERSION,
        "model_version": f"{version}-onnx",
        "storage_path": f"{s3_model_bucket}/{s3_onnx}",
        "description": "Dense Neural Network trained on music data (ONNX)",
        "metadata": {
                    "license": "apache-2.0"
                },
        "upload_parms": s3_upload_params_onnx
    }
]

In [ ]:
# register models
for model in models:
    print(f"Registering: {model.get('model_version')}...")
    register(model_name=model.get('model_name'),
             data_path=model.get('data_path'),
             model_format_name=model.get('model_format_name'),
             author=model.get('author'),
             model_format_version=model.get('model_format_version'),
             model_version=model.get('model_version'),
             storage_path=model.get('storage_path'),
             description=model.get('description'),
             metadata=model.get('metadata'),
             upload_parms=model.get('upload_parms'))